In [2]:
!pip install pytorch_pretrained_bert
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
os.chdir('/content/drive/My Drive/bert_dataset/assignment')
os.getcwd()
os.listdir()

['train_final_trans.txt',
 'test_final_trans.txt',
 'test.txt',
 'train.txt',
 'myfile.txt',
 'final.txt',
 'ep_10.bin',
 'ep_100.bin']

In [0]:
train_tweets=[]
train_tags=[]
test_tweets=[]
test_tags=[]
d={'negative':0,'neutral':1,'positive':2}
with open('train_final_trans.txt','r') as f:
    k = f.readlines()
    for i in k:
        l=i.strip().split()
        train_tweets.append(' '.join(l[1:-1]))
        train_tags.append(d[l[-1]])
with open('test_final_trans.txt','r') as f:
    k = f.readlines()
    for i in k:
        l=i.strip().split()
        test_tweets.append(' '.join(l[1:-1]))
        test_tags.append(d[l[-1]])

In [5]:
import torch
import json
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

# specify GPU device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

Using TensorFlow backend.


'Tesla K80'

In [6]:
sentences = ["[CLS] " + query + " [SEP]" for query in train_tweets]
print(sentences[0])
print(len(sentences))

# Tokenize with BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print ("Tokenize the first sentence:")
print (tokenized_texts[0])

[CLS] pakistan splotch tauq pakistan israel limitless other kerta him palestine Says occupied palestine [SEP]
15130
Tokenize the first sentence:
['[CLS]', 'pakistan', 'sp', '##lot', '##ch', 'tau', '##q', 'pakistan', 'israel', 'limit', '##less', 'other', 'ke', '##rta', 'him', 'palestine', 'says', 'occupied', 'palestine', '[SEP]']


In [7]:
MAX_LEN = 128
# Pad our input tokens
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
# Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

attention_masks = []
# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)


y_train=np.array(train_tags)
y_train[2]

2

In [0]:
train_inputs = torch.tensor(input_ids)
#validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(y_train)
#validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(attention_masks)
#validation_masks = torch.tensor(validation_masks)

# Select a batch size for training. 
batch_size = 32

# Create an iterator of our data with torch DataLoader 
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_dataloader = DataLoader(train_data, batch_size=batch_size)

In [9]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)
model.cuda()


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
   

In [19]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=2e-5,
                     warmup=.1)

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)
  
# Store our loss and accuracy for plotting
train_loss_set = []
# Number of training epochs 
epochs = 10

# BERT training loop
for _ in trange(epochs, desc="Epoch"):  
  
  ## TRAINING
  
  # Set our model to training mode
  model.train()  
  # Tracking variables
  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0
  # Train the data for one epoch
  for step, batch in enumerate(train_dataloader):
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    #b_input_ids=b_input_ids.to(device)
    #b_input_mask=b_input_mask.to(device)
    b_labels=b_labels.long()
    #print(device)
    # Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()
    # Forward pass

    loss = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
    train_loss_set.append(loss.item())    
    # Backward pass
    loss.backward()
    # Update parameters and take a step using the computed gradient
    optimizer.step()
    # Update tracking variables
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1
    if nb_tr_steps % 50 == 0:


      print(nb_tr_steps)
    
  print("Train loss: {}".format(tr_loss/nb_tr_steps))
       
  ## VALIDATION



# plot training performance
plt.figure(figsize=(15,8))
plt.title("Training loss")
plt.xlabel("Batch")
plt.ylabel("Loss")
plt.plot(train_loss_set)
plt.show()

t_total value of -1 results in schedule not being applied

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

50
100
150
200
250
300
350
400
450



Epoch:  10%|█         | 1/10 [11:45<1:45:45, 705.06s/it]

Train loss: 0.6907598155434489
50
100
150
200
250
300
350
400
450



Epoch:  20%|██        | 2/10 [23:31<1:34:03, 705.40s/it]

...


Epoch:  30%|███       | 3/10 [35:17<1:22:19, 705.61s/it]

Train loss: 0.5025176505615293
50
100
150
200
250
300
350
400
450



Epoch:  40%|████      | 4/10 [47:05<1:10:38, 706.34s/it]

Train loss: 0.4421404496654396
50


KeyboardInterrupt: ignored

In [20]:
sentences = ["[CLS] " + query + " [SEP]" for query in test_tweets]
print(sentences[0])
print(len(sentences))

# Tokenize with BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print ("Tokenize the first sentence:")
print (tokenized_texts[0])

MAX_LEN = 128
# Pad our input tokens
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
# Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
print("no of input",input_ids.shape)

attention_masks = []
# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

y_train=np.array(test_tags)
print(y_train.shape)

train_inputs = torch.tensor(input_ids)
#validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(y_train)
#validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(attention_masks)
#validation_masks = torch.tensor(validation_masks)

# Select a batch size for training. 
batch_size = 32

# Create an iterator of our data with torch DataLoader 
train_data = TensorDataset(train_inputs, train_masks, train_labels)
#train_sampler = RandomSampler(train_data)
prediction_dataloader = DataLoader(train_data, batch_size=batch_size)


[CLS] love looks good maddie ako clove young sobrang spice how may zolo soya before with the past mayo lowkey [SEP]
1868
Tokenize the first sentence:
['[CLS]', 'love', 'looks', 'good', 'maddie', 'ak', '##o', 'cl', '##ove', 'young', 'sob', '##rang', 'spice', 'how', 'may', 'z', '##olo', 'soy', '##a', 'before', 'with', 'the', 'past', 'mayo', 'low', '##key', '[SEP]']
no of input (1868, 128)
(1868,)


In [21]:
model=model.cuda()
model.eval()
# Tracking variables 
predictions , true_labels = [], []
# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  # Telling the model not to compute or store gradients, saving memory and speeding up prediction
  with torch.no_grad():
    # Forward pass, calculate logit predictions
    logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)
  
# Import and evaluate each test batch using Matthew's correlation coefficient
from sklearn.metrics import matthews_corrcoef
matthews_set = []
for i in range(len(true_labels)):
  matthews = matthews_corrcoef(true_labels[i],
                 np.argmax(predictions[i], axis=1).flatten())
  matthews_set.append(matthews)
  
# Flatten the predictions and true values for aggregate Matthew's evaluation on the whole dataset
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
flat_true_labels = [item for sublist in true_labels for item in sublist]

print('Classification accuracy using BERT Fine Tuning: {0:0.2%}'.format(matthews_corrcoef(flat_true_labels, flat_predictions)))

Classification accuracy using BERT Fine Tuning: 34.12%


In [22]:
from sklearn.metrics import classification_report

print(classification_report(flat_true_labels,flat_predictions))

              precision    recall  f1-score   support

           0       0.55      0.67      0.61       532
           1       0.54      0.54      0.54       754
           2       0.61      0.49      0.55       582

    accuracy                           0.56      1868
   macro avg       0.57      0.57      0.57      1868
weighted avg       0.57      0.56      0.56      1868

